In [229]:
# For data cleaning and EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import random as rnd
import plotly.express as px 
import datetime

# To print multiple outputs in a single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [230]:
df = pd.read_csv('../data/CombinedData/cleaned_air_quality.csv')

In [231]:
df.head()

,Timestamp,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NH3 (µg/m³),SO2 (µg/m³),Ozone (µg/m³),Benzene (µg/m³),Toluene (µg/m³),...,VWS (m/s),station,hour,day,month,year,Wind Impact,NOx (µg/m³),CO (µg/m³),BP (hPa)
0,2021-01-01 00:00:00,21.20,219.03,4.19,11.28,2.62,8.32,0.93,0.00,2.15,...,-0.44,alandur,0,1,1,2021,37.2058,29.1024,700.0,1016.086959
1,2021-01-01 01:00:00,21.20,219.03,4.27,11.34,2.62,8.14,0.49,0.00,0.00,...,-0.44,alandur,1,1,1,2021,5.4760,29.1588,700.0,1015.793650
2,2021-01-01 02:00:00,19.44,219.03,4.27,11.29,2.62,8.24,0.74,0.00,0.00,...,-0.44,alandur,2,1,1,2021,13.3056,29.2716,700.0,1015.313691
3,2021-01-01 03:00:00,18.66,219.03,4.22,11.30,2.62,8.21,0.67,1.33,0.00,...,-0.44,alandur,3,1,1,2021,6.4614,29.1588,700.0,1015.007050
4,2021-01-01 04:00:00,17.44,219.03,4.24,11.24,2.62,8.30,1.14,1.88,0.00,...,-0.44,alandur,4,1,1,2021,4.2174,29.2152,700.0,1015.140372


In [232]:
pollutants = ['PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NOx (µg/m³)',
                'CO (µg/m³)', 'SO2 (µg/m³)', 'Ozone (µg/m³)']
df = df[['Timestamp', 'year', 'month', 'day', 'station'] + pollutants]

In [233]:
df

,Timestamp,year,month,day,station,PM2.5 (µg/m³),PM10 (µg/m³),NOx (µg/m³),CO (µg/m³),SO2 (µg/m³),Ozone (µg/m³)
0,2021-01-01 00:00:00,2021,1,1,alandur,21.200000,219.0300,29.102400,700.0,8.320000,0.930000
1,2021-01-01 01:00:00,2021,1,1,alandur,21.200000,219.0300,29.158800,700.0,8.140000,0.490000
2,2021-01-01 02:00:00,2021,1,1,alandur,19.440000,219.0300,29.271600,700.0,8.240000,0.740000
3,2021-01-01 03:00:00,2021,1,1,alandur,18.660000,219.0300,29.158800,700.0,8.210000,0.670000
4,2021-01-01 04:00:00,2021,1,1,alandur,17.440000,219.0300,29.215200,700.0,8.300000,1.140000
...,...,...,...,...,...,...,...,...,...,...,...
92947,2023-12-31 19:00:00,2023,12,31,velachery,22.172500,69.8975,36.923200,1670.0,11.097500,27.360000
92948,2023-12-31 20:00:00,2023,12,31,velachery,26.697500,72.8500,37.534200,1405.0,11.090000,28.410000
92949,2023-12-31 21:00:00,2023,12,31,velachery,27.570000,79.3550,28.594800,1317.5,11.150000,27.705000
92950,2023-12-31 22:00:00,2023,12,31,velachery,26.042500,81.6225,28.834500,1477.5,11.307500,25.022500


In [234]:
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN
    


from decimal import Decimal

# AQI Sub-index Calculation Functions
def get_PM25_subindex(x):
    x = Decimal(x)
    if x <= 30: return round(x * 50 / 30)
    elif x <= 60: return round(50 + (x - 30) * 50 / 30)
    elif x <= 90: return round(100 + (x - 60) * 100 / 30)
    elif x <= 120: return round(200 + (x - 90) * 100 / 30)
    elif x <= 250: return round(300 + (x - 120) * 100 / 130)
    else: return round(400 + (x - 250) * 100 / 130)

def get_PM10_subindex(x):
    x = Decimal(x)
    if x <= 50: return round(x * 50 / 50)
    elif x <= 100: return round(50 + (x - 50) * 50 / 50)
    elif x <= 250: return round(100 + (x - 100) * 100 / 150)
    elif x <= 350: return round(200 + (x - 250) * 100 / 100)
    elif x <= 430: return round(300 + (x - 350) * 100 / 80)
    else: return round(400 + (x - 430) * 100 / 80)

def get_SO2_subindex(x):
    x = Decimal(x)
    if x <= 40: return round(x * 50 / 40)
    elif x <= 80: return round(50 + (x - 40) * 50 / 40)
    elif x <= 380: return round(100 + (x - 80) * 100 / 300)
    elif x <= 800: return round(200 + (x - 380) * 100 / 420)
    elif x <= 1600: return round(300 + (x - 800) * 100 / 800)
    else: return round(400 + (x - 1600) * 100 / 800)

def get_NOx_subindex(x):
    x = Decimal(x)
    if x <= 40: return round(x * 50 / 40)
    elif x <= 80: return round(50 + (x - 40) * 50 / 40)
    elif x <= 180: return round(100 + (x - 80) * 100 / 100)
    elif x <= 280: return round(200 + (x - 180) * 100 / 100)
    elif x <= 400: return round(300 + (x - 280) * 100 / 120)
    else: return round(400 + (x - 400) * 100 / 120)

def get_CO_subindex(x):
    x = Decimal(x) / 1000  # Convert µg/m³ to mg/m³
    if x <= 1: return round(x * 50 / 1)
    elif x <= 2: return round(50 + (x - 1) * 50 / 1)
    elif x <= 10: return round(100 + (x - 2) * 100 / 8)
    elif x <= 17: return round(200 + (x - 10) * 100 / 7)
    elif x <= 34: return round(300 + (x - 17) * 100 / 17)
    else: return round(400 + (x - 34) * 100 / 17)

def get_O3_subindex(x):
    x = Decimal(x)
    if x <= 50: return round(x * 50 / 50)
    elif x <= 100: return round(50 + (x - 50) * 50 / 50)
    elif x <= 168: return round(100 + (x - 100) * 100 / 68)
    elif x <= 208: return round(200 + (x - 168) * 100 / 40)
    elif x <= 748: return round(300 + (x - 208) * 100 / 540)
    else: return round(400 + (x - 748) * 100 / 540)


In [235]:
df['datetime'] = pd.to_datetime(df['Timestamp'])
df.drop(columns=['Timestamp'], inplace=True)

In [236]:
df["NOx_SubIndex"] = df["NOx (µg/m³)"].apply(lambda x: get_NOx_subindex(x))
df["SO2_SubIndex"] = df["SO2 (µg/m³)"].apply(lambda x: get_SO2_subindex(x))
df["PM2.5_SubIndex"] = df["PM2.5 (µg/m³)"].apply(lambda x: get_PM25_subindex(x))
df["CO_SubIndex"] = (df["CO (µg/m³)"]).apply(lambda x: get_CO_subindex(x))
df["AQI_calculated"] = round(df[["PM2.5_SubIndex", "SO2_SubIndex", "NOx_SubIndex", "CO_SubIndex"]].max(axis=1))


In [237]:
# Ensure datetime column is in datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Calculate 24-hour rolling average for PM2.5
df["PM2.5_24_avg"] = df.groupby("station")["PM2.5 (µg/m³)"].rolling(window=24, min_periods=16).mean().reset_index(level=0, drop=True)

# Display the dataframe
df



,year,month,day,station,PM2.5 (µg/m³),PM10 (µg/m³),NOx (µg/m³),CO (µg/m³),SO2 (µg/m³),Ozone (µg/m³),datetime,NOx_SubIndex,SO2_SubIndex,PM2.5_SubIndex,CO_SubIndex,AQI_calculated,PM2.5_24_avg
0,2021,1,1,alandur,21.200000,219.0300,29.102400,700.0,8.320000,0.930000,2021-01-01 00:00:00,36,10,35,35,36,NaN
1,2021,1,1,alandur,21.200000,219.0300,29.158800,700.0,8.140000,0.490000,2021-01-01 01:00:00,36,10,35,35,36,NaN
2,2021,1,1,alandur,19.440000,219.0300,29.271600,700.0,8.240000,0.740000,2021-01-01 02:00:00,37,10,32,35,37,NaN
3,2021,1,1,alandur,18.660000,219.0300,29.158800,700.0,8.210000,0.670000,2021-01-01 03:00:00,36,10,31,35,36,NaN
4,2021,1,1,alandur,17.440000,219.0300,29.215200,700.0,8.300000,1.140000,2021-01-01 04:00:00,37,10,29,35,37,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92947,2023,12,31,velachery,22.172500,69.8975,36.923200,1670.0,11.097500,27.360000,2023-12-31 19:00:00,46,14,37,84,84,27.610903
92948,2023,12,31,velachery,26.697500,72.8500,37.534200,1405.0,11.090000,28.410000,2023-12-31 20:00:00,47,14,44,70,70,27.197257
92949,2023,12,31,velachery,27.570000,79.3550,28.594800,1317.5,11.150000,27.705000,2023-12-31 21:00:00,36,14,46,66,66,26.784757
92950,2023,12,31,velachery,26.042500,81.6225,28.834500,1477.5,11.307500,25.022500,2023-12-31 22:00:00,36,14,43,74,74,26.505590


In [238]:
# Calculate the 8-hour rolling average for 'CO'
df['CO_8_avg'] = df['CO (µg/m³)'].rolling(window=8, min_periods=1).mean()

# Display the first 20 rows to see the result
df[['datetime', 'CO (µg/m³)', 'CO_8_avg']].head(20)


,datetime,CO (µg/m³),CO_8_avg
0,2021-01-01 00:00:00,700.0,700.000000
1,2021-01-01 01:00:00,700.0,700.000000
2,2021-01-01 02:00:00,700.0,700.000000
3,2021-01-01 03:00:00,700.0,700.000000
4,2021-01-01 04:00:00,700.0,700.000000
5,2021-01-01 05:00:00,700.0,700.000000
6,2021-01-01 06:00:00,710.0,701.428571
7,2021-01-01 07:00:00,710.0,702.500000
8,2021-01-01 08:00:00,700.0,702.500000
9,2021-01-01 09:00:00,700.0,702.500000


In [239]:
# Drop unnecessary columns
df.drop(['PM2.5_SubIndex', 'CO_SubIndex'], axis=1, inplace=True)

# Calculate subindices for 24-hour PM2.5 average and 8-hour CO average
df['PM2.5_24_avg_Sub_Index'] = df['PM2.5_24_avg'].apply(lambda x: get_PM25_subindex(x) if not np.isnan(x) else np.nan)
df['CO_8_avg_Sub_Index'] = df['CO_8_avg'].apply(lambda x: get_CO_subindex(x) if not np.isnan(x) else np.nan)

# Recalculate AQI
df["AQI_calculated"] = round(df[["PM2.5_24_avg_Sub_Index", "SO2_SubIndex", "CO_8_avg_Sub_Index", 'NOx_SubIndex']].max(axis=1))

# Display the dataframe
df


,year,month,day,station,PM2.5 (µg/m³),PM10 (µg/m³),NOx (µg/m³),CO (µg/m³),SO2 (µg/m³),Ozone (µg/m³),datetime,NOx_SubIndex,SO2_SubIndex,AQI_calculated,PM2.5_24_avg,CO_8_avg,PM2.5_24_avg_Sub_Index,CO_8_avg_Sub_Index
0,2021,1,1,alandur,21.200000,219.0300,29.102400,700.0,8.320000,0.930000,2021-01-01 00:00:00,36,10,36.0,NaN,700.0000,NaN,35
1,2021,1,1,alandur,21.200000,219.0300,29.158800,700.0,8.140000,0.490000,2021-01-01 01:00:00,36,10,36.0,NaN,700.0000,NaN,35
2,2021,1,1,alandur,19.440000,219.0300,29.271600,700.0,8.240000,0.740000,2021-01-01 02:00:00,37,10,37.0,NaN,700.0000,NaN,35
3,2021,1,1,alandur,18.660000,219.0300,29.158800,700.0,8.210000,0.670000,2021-01-01 03:00:00,36,10,36.0,NaN,700.0000,NaN,35
4,2021,1,1,alandur,17.440000,219.0300,29.215200,700.0,8.300000,1.140000,2021-01-01 04:00:00,37,10,37.0,NaN,700.0000,NaN,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92947,2023,12,31,velachery,22.172500,69.8975,36.923200,1670.0,11.097500,27.360000,2023-12-31 19:00:00,46,14,46.0,27.610903,912.8125,46.0,46
92948,2023,12,31,velachery,26.697500,72.8500,37.534200,1405.0,11.090000,28.410000,2023-12-31 20:00:00,47,14,49.0,27.197257,972.1875,45.0,49
92949,2023,12,31,velachery,27.570000,79.3550,28.594800,1317.5,11.150000,27.705000,2023-12-31 21:00:00,36,14,52.0,26.784757,1043.4375,45.0,52
92950,2023,12,31,velachery,26.042500,81.6225,28.834500,1477.5,11.307500,25.022500,2023-12-31 22:00:00,36,14,57.0,26.505590,1131.8750,44.0,57


In [240]:
df["AQI_Bucket"] = df["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))

In [241]:
df

,year,month,day,station,PM2.5 (µg/m³),PM10 (µg/m³),NOx (µg/m³),CO (µg/m³),SO2 (µg/m³),Ozone (µg/m³),datetime,NOx_SubIndex,SO2_SubIndex,AQI_calculated,PM2.5_24_avg,CO_8_avg,PM2.5_24_avg_Sub_Index,CO_8_avg_Sub_Index,AQI_Bucket
0,2021,1,1,alandur,21.200000,219.0300,29.102400,700.0,8.320000,0.930000,2021-01-01 00:00:00,36,10,36.0,NaN,700.0000,NaN,35,Good
1,2021,1,1,alandur,21.200000,219.0300,29.158800,700.0,8.140000,0.490000,2021-01-01 01:00:00,36,10,36.0,NaN,700.0000,NaN,35,Good
2,2021,1,1,alandur,19.440000,219.0300,29.271600,700.0,8.240000,0.740000,2021-01-01 02:00:00,37,10,37.0,NaN,700.0000,NaN,35,Good
3,2021,1,1,alandur,18.660000,219.0300,29.158800,700.0,8.210000,0.670000,2021-01-01 03:00:00,36,10,36.0,NaN,700.0000,NaN,35,Good
4,2021,1,1,alandur,17.440000,219.0300,29.215200,700.0,8.300000,1.140000,2021-01-01 04:00:00,37,10,37.0,NaN,700.0000,NaN,35,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92947,2023,12,31,velachery,22.172500,69.8975,36.923200,1670.0,11.097500,27.360000,2023-12-31 19:00:00,46,14,46.0,27.610903,912.8125,46.0,46,Good
92948,2023,12,31,velachery,26.697500,72.8500,37.534200,1405.0,11.090000,28.410000,2023-12-31 20:00:00,47,14,49.0,27.197257,972.1875,45.0,49,Good
92949,2023,12,31,velachery,27.570000,79.3550,28.594800,1317.5,11.150000,27.705000,2023-12-31 21:00:00,36,14,52.0,26.784757,1043.4375,45.0,52,Satisfactory
92950,2023,12,31,velachery,26.042500,81.6225,28.834500,1477.5,11.307500,25.022500,2023-12-31 22:00:00,36,14,57.0,26.505590,1131.8750,44.0,57,Satisfactory


In [242]:
# Fill NaN values in PM2.5_24_avg column using forward fill
df['PM2.5_24_avg'].fillna(method='ffill', inplace=True)

# Remove rows with any remaining NaN values
df.dropna(inplace=True)

# Display the dataframe to check the result
df

/tmp/ipykernel_3171/3761809118.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['PM2.5_24_avg'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_3171/3761809118.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PM2.5_24_avg'].fillna(method='ffill', inplace=True)


,year,month,day,station,PM2.5 (µg/m³),PM10 (µg/m³),NOx (µg/m³),CO (µg/m³),SO2 (µg/m³),Ozone (µg/m³),datetime,NOx_SubIndex,SO2_SubIndex,AQI_calculated,PM2.5_24_avg,CO_8_avg,PM2.5_24_avg_Sub_Index,CO_8_avg_Sub_Index,AQI_Bucket
15,2021,1,1,alandur,28.000000,219.0300,29.177600,710.0,8.220000,14.220000,2021-01-01 15:00:00,36,10,36.0,20.414375,697.5000,34.0,35,Good
16,2021,1,1,alandur,25.520000,219.0300,29.234000,720.0,8.060000,8.880000,2021-01-01 16:00:00,37,10,37.0,20.714706,700.0000,35.0,35,Good
17,2021,1,1,alandur,25.230000,219.0300,29.328000,700.0,8.190000,2.940000,2021-01-01 17:00:00,37,10,37.0,20.965556,700.0000,35.0,35,Good
18,2021,1,1,alandur,25.580000,219.0300,29.140000,700.0,8.330000,1.100000,2021-01-01 18:00:00,36,10,36.0,21.208421,701.2500,35.0,35,Good
19,2021,1,1,alandur,24.770000,219.0300,29.177600,700.0,8.060000,1.100000,2021-01-01 19:00:00,36,10,36.0,21.386500,701.2500,36.0,35,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92947,2023,12,31,velachery,22.172500,69.8975,36.923200,1670.0,11.097500,27.360000,2023-12-31 19:00:00,46,14,46.0,27.610903,912.8125,46.0,46,Good
92948,2023,12,31,velachery,26.697500,72.8500,37.534200,1405.0,11.090000,28.410000,2023-12-31 20:00:00,47,14,49.0,27.197257,972.1875,45.0,49,Good
92949,2023,12,31,velachery,27.570000,79.3550,28.594800,1317.5,11.150000,27.705000,2023-12-31 21:00:00,36,14,52.0,26.784757,1043.4375,45.0,52,Satisfactory
92950,2023,12,31,velachery,26.042500,81.6225,28.834500,1477.5,11.307500,25.022500,2023-12-31 22:00:00,36,14,57.0,26.505590,1131.8750,44.0,57,Satisfactory


In [243]:

df.to_csv('../data/CombinedData/combined_aqi.csv', index=False)